# Data Warehouse with Redshift - Create Tables
Use this notebook to develop the code before completing the `create_tables.py` file to create each of your tables.

>
> **Stephanie Anderton**  
> DEND Project \#3  
> May 28, 2019
>

In [1]:
import configparser
import psycopg2
from sql_queries import create_table_queries, drop_table_queries
import time
import re
import mylib
from mylib import logger

In [2]:
def drop_tables(cur, conn):
    for query in drop_table_queries:
        # the table name is the 5th word in the query string
        table = re.findall(r'\w+', query)[4]
        logger.info('delete table [ {} ]'.format(table))
        try:
            cur.execute(query)
            conn.commit()
        except psycopg2.Error as e: 
            logger.info('Error: Dropping table [ {} ]'.format(table))
            print(e)
            print(query)


In [3]:
def create_tables(cur, conn):
    for query in create_table_queries:
        # the table name is the 6th word in the query string
        table = re.findall(r'\w+', query)[5]
        logger.info('create table [ {} ]'.format(table))
        try:
            cur.execute(query)
            conn.commit()
        except psycopg2.Error as e: 
            logger.info('Error: Creating table [ {} ]'.format(table))
            print(e)
            print(query)


## Main()

In [4]:
logger.info('---[ Create Tables ]---')
mylib.log_timestamp(config)
print("Logfile:  " + mylib.log_file_name())

# read config parameters for database connection string
config = configparser.ConfigParser()
config.read('dwh.cfg')

try:
    conn_string = "host={} dbname={} user={} password={} port={}"
    conn_string = conn_string.format(*config['CLUSTER'].values())
    conn = psycopg2.connect( conn_string )
    cur = conn.cursor()
    print(conn_string)

except Exception as e:
    logger.info("Error: Could not make connection to the sparkify DB")
    print(e)

logger.info('Drop existing tables...')
drop_tables(cur, conn)

logger.info('Create tables...')
create_tables(cur, conn)

conn.close()
logger.info('DB connection closed')


host=dwhcluster.cbsjbxldkge8.us-west-2.redshift.amazonaws.com dbname=sparkify user=dwhuser password=Passw0rd port=5439
